In [1]:
import numpy as np
import pandas as pd
import rasterio
import rioxarray as rxr
import xarray as xr
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
def sds(hdf_path, short_name):
    # Build the GDAL subdataset path; grid name is 'grid1km' per your gdalinfo
    return f'HDF4_EOS:EOS_GRID:"{hdf_path}":grid1km:{short_name}'

def bit_slice(arr, start, end):
    mask = (1 << (end - start + 1)) - 1
    return (arr >> start) & mask

def mask_from_qa(qa_uint16):
    # MAIAC strict mask
    cloud = bit_slice(qa_uint16, 0, 2)     # 001 clear
    lws   = bit_slice(qa_uint16, 3, 4)     # 00 land
    adj   = bit_slice(qa_uint16, 5, 7)     # 000 normal
    aodq  = bit_slice(qa_uint16, 8, 11)    # 0000 best
    glint = bit_slice(qa_uint16, 12, 12)   # 0 no glint
    return ((cloud == 0b001) | (cloud == 0b010)) & \
            (lws == 0b00) & \
            ((adj == 0b000) | (adj == 0b011)) & \
            (aodq == 0b0000) & \
            (glint == 0)

def read_masked_daily_aod055(hdf_path):
    # Read as DataArrays with a "band" dimension of length up to 4
    aod = rxr.open_rasterio(sds(hdf_path, "Optical_Depth_055"), masked=True).astype("float32")
    qa  = rxr.open_rasterio(sds(hdf_path, "AOD_QA"),             masked=True).astype("uint16")

    # Scale AOD (INT16 * 0.001). Keep shape [band, y, x].
    aod = aod * 0.001

    # Apply QA per band
    good = mask_from_qa(qa.values)  # shape [band, y, x]
    aod.data = np.where(good, aod.values, np.nan)

    # Reduce across multiple overpasses (bands) to a single daily raster (median is robust)
    # (xarray names the dim "band" by default)
    aod_daily = aod.median(dim="band")#, skipna=True)

    # Ensure NoData is NaN
    aod_daily = aod_daily.rename("AOD055")
    aod_daily.rio.write_nodata(np.nan, inplace=True)
    return aod_daily

In [3]:
def mosaic_and_reproject(list_of_dss):
    # Mosaic in native CRS
    '''
    Mosaic multiple DataArrays by averaging overlapping pixels.
    Assumes all DataArrays have the same CRS and resolution.
    list_if_dss: list of xarray.DataArray
    returns: xarray.DataArray
    '''
    stack = xr.concat(list_of_dss, dim="stack")
    mosaic = stack.mean("stack", skipna=True)

    # Reproject to WGS84 (≈1 km at equator = 30 arcsec)
    mosaic_wgs84 = mosaic.rio.reproject(
        "EPSG:4326",
        resolution=0.01, #0.00926625433
        resampling=rasterio.enums.Resampling.bilinear,
        nodata=np.nan
    )
    return mosaic_wgs84

In [4]:
def convert_names_to_dates(files):
    year = []
    day = []
    #hr = []
    #min = []
    dts = []
    for file in files:
        dts.append(file[9:16])
        year.append(int(file[9:13]))
        day.append(int(file[13:16]))
        #hr.append(int(file[18:20]))
        #min.append(int(file[20:22]))
    files = pd.DataFrame(files, columns=['Files'])
    dts = pd.DataFrame(dts, columns=['MODIS_Dates'])
    year = pd.DataFrame(year, columns=['Year'])
    day = pd.DataFrame(day, columns=['Day'])
    #hr = pd.DataFrame(hr, columns=['Hour'])
    #min = pd.DataFrame(min, columns=['Minute'])
    dtdf = pd.concat([files, dts, year, day], axis=1)
    dtdf['datetime'] = pd.to_datetime(dtdf['Year'] * 1000 + dtdf['Day'], format='%Y%j')
    dtdf.sort_values(by=['datetime'], inplace=True, ascending=True)
    dtdf.reset_index(drop=True, inplace=True)
    return dtdf

In [5]:
def hdf_to_netcdf(folder, minx, miny, maxx, maxy):
    files = os.listdir(folder)
    df = convert_names_to_dates(files)
    for date in df['datetime'].unique():
        dff = df[df['datetime'] == date]
        dss = [read_masked_daily_aod055(folder + file) for file in dff['Files'] if file.endswith('.hdf')]
        s = mosaic_and_reproject(dss).rio.clip_box(minx=minx-1e-16, miny=miny-1e-16, maxx=maxx+1e-16, maxy=maxy+1e-16)
        s['x'] = np.round(s['x'], 2)
        s['y'] = np.round(s['y'], 2)
        ds_date = s.to_dataset().expand_dims(time=[date]).assign_coords(time=("time", [date]))
        if date == df['datetime'].unique()[0]:
            final_ds = ds_date
        else:
            final_ds = xr.concat([final_ds, ds_date], dim='time')

    return final_ds

In [6]:
minx=35.5
miny=31.0
maxx=37.4
maxy=32.2

In [7]:
os.listdir('./HDF files/')

['MCD19A2.A2024001.h20v05.061.2024004181335.hdf',
 'MCD19A2.A2024001.h21v05.061.2024004182223.hdf',
 'MCD19A2.A2024002.h20v05.061.2024005025822.hdf',
 'MCD19A2.A2024002.h21v05.061.2024005030454.hdf',
 'MCD19A2.A2024003.h20v05.061.2024005144949.hdf',
 'MCD19A2.A2024003.h21v05.061.2024005150129.hdf',
 'MCD19A2.A2024004.h20v05.061.2024006003414.hdf',
 'MCD19A2.A2024004.h21v05.061.2024006005356.hdf',
 'MCD19A2.A2024005.h20v05.061.2024009021136.hdf',
 'MCD19A2.A2024005.h21v05.061.2024008224436.hdf',
 'MCD19A2.A2024006.h20v05.061.2024009051050.hdf',
 'MCD19A2.A2024006.h21v05.061.2024009024756.hdf',
 'MCD19A2.A2024007.h20v05.061.2024009082111.hdf',
 'MCD19A2.A2024007.h21v05.061.2024009045207.hdf',
 'MCD19A2.A2024008.h20v05.061.2024009203044.hdf',
 'MCD19A2.A2024008.h21v05.061.2024009210450.hdf',
 'MCD19A2.A2024009.h20v05.061.2024011162450.hdf',
 'MCD19A2.A2024009.h21v05.061.2024011164357.hdf',
 'MCD19A2.A2024010.h20v05.061.2024011214808.hdf',
 'MCD19A2.A2024010.h21v05.061.2024011212757.hdf',


In [8]:
folder = './d2/'
hdf_to_netcdf(folder, minx, miny, maxx, maxy)

<xarray.Dataset> Size: 465kB
Dimensions:      (time: 5, y: 121, x: 191)
Coordinates:
  * time         (time) datetime64[ns] 40B 2024-01-01 2024-01-02 ... 2024-01-05
  * y            (y) float64 968B 32.2 32.19 32.18 32.17 ... 31.02 31.01 31.0
  * x            (x) float64 2kB 35.5 35.51 35.52 35.53 ... 37.38 37.39 37.4
    spatial_ref  int64 8B 0
Data variables:
    AOD055       (time, y, x) float32 462kB 0.2082 0.2117 ... 0.06581 0.06676

In [9]:
folder = './HDF files/'
nc = hdf_to_netcdf(folder, minx, miny, maxx, maxy)

In [10]:
nc

<xarray.Dataset> Size: 36MB
Dimensions:      (time: 385, y: 121, x: 191)
Coordinates:
  * time         (time) datetime64[ns] 3kB 2024-01-01 2024-01-02 ... 2025-01-20
  * y            (y) float64 968B 32.2 32.19 32.18 32.17 ... 31.02 31.01 31.0
  * x            (x) float64 2kB 35.5 35.51 35.52 35.53 ... 37.38 37.39 37.4
    spatial_ref  int64 8B 0
Data variables:
    AOD055       (time, y, x) float32 36MB 0.2082 0.2117 ... 0.06404 0.06652

In [11]:
nc.to_netcdf('./AOD nc/2024.nc')